In [ ]:
%matplotlib inline
import numpy as np
import torch
import scipy
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm

from scipy.stats import levy_stable
from scipy.special import gamma as gamma_func

In [ ]:
class Fractional_Langevin_MonteCarlo():
    def __init__(self, m=None):
        self.m = m

    def phi(self, U):
        # unnormalized target density : exp(-U(X))
        return torch.exp(-U)

    def drift_hat(self, x, alpha, U):
        # simplified multidimensional drift
        # approximate fractional derivative by using only the first term of centered difference operator

        x.requires_grad_()
        u = U(x)
        grad = torch.autograd.grad(u, x)[0]
        
        def c_alpha(alpha):
            return gamma_func(alpha-1) / gamma_func(alpha/2)**2

        return -c_alpha(alpha) * grad
    
    def FLA(self, U, alpha, N, a_eta, b_eta, step_size, dim):
        # Fractional Langevin Algorithm(FLA)
        # when alpha = 2, FLA is same as ULA
        burn_in = 5000
        x_0 = torch.randn(1, dim).double()

        Levy_motions = levy_stable.rvs(alpha, 0, size=dim * (N + burn_in))
        Levy_motions = torch.from_numpy(Levy_motions.reshape((Levy_motions.shape[0] // dim, dim)))

        x_i = x_0
        samples = []

        for i in tqdm.tqdm(range(N + burn_in)):
            #step_size = (a_eta/(i+1))**b_eta
            step_size = step_size
            b = self.drift_hat(x_i, alpha, U)
            x_i = x_i.detach() + step_size * b + step_size ** (1./alpha) * Levy_motions[i]

            if dim > 1:
                samples.append(x_i.detach().numpy())
            else:
                samples.append(x_i.detach().numpy().squeeze())

        if dim > 1:
            return np.concatenate(samples, 0)[burn_in:]
        else:
            return samples[burn_in:]

## Experiment from paper

In [ ]:
FLMC = Fractional_Langevin_MonteCarlo()

In [ ]:
# Section 4. Experiment with double-well potential
def double_well_potential(x):
    u = (x+5)*(x+1) * (x-1.02) * (x-5) / 10 + 0.5
    return u

def double_well_density_np(x):
    u = double_well_potential(x)
    return np.exp(-u)

def double_well_density(x):
    u = double_well_potential(x)
    return torch.exp(-u)

In [ ]:
# Visualize Figure 2 (top) in paper
x = np.linspace(-6, 6, 1000)
y = double_well_density_np(x)

plt.figure(figsize = (10,2))
plt.plot(x, y)
plt.title('Figure 2 (Top)')
plt.show()

In [ ]:
# Visualize Figure 2 (middle) in paper
# empirical distribution obtained via ULA (corresponds to FLA with alpha = 2)
samples = FLMC.FLA(U=double_well_potential, alpha=2, N=10000, a_eta=1e-7, b_eta=1, step_size=0.01, dim=1)

plt.figure(figsize = (10,2))
plt.hist(samples, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Figure 2 (Middle)')
plt.show()

samples = FLMC.FLA(U=double_well_potential, alpha=1.7, N=10000, a_eta=1e-7, b_eta=1, step_size=0.01, dim=1)

plt.figure(figsize = (10,2))
plt.hist(samples, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Figure 2 (Middle)')
plt.show()


## Multi-dimentional Case

In [ ]:
def density1_np(z):
    z1, z2 = z[:, 0], z[:, 1]
    norm = np.sqrt(z1 ** 2 + z2 ** 2)
    exp1 = np.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = np.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - np.log(exp1 + exp2)
    return np.exp(-u)

def potential1(z):
    z = z.view(-1, 2).double()
    z1, z2 = z[:, 0], z[:, 1]
    norm = torch.norm(z, p=2, dim=1)
    exp1 = torch.exp(-0.5 * ((z1 - 2) / 0.6) ** 2)
    exp2 = torch.exp(-0.5 * ((z1 + 2) / 0.6) ** 2)
    u = 0.5 * ((norm - 2) / 0.4) ** 2 - torch.log(exp1 + exp2)
    return u


In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = density1_np(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.title('Density #1')
plt.show()

In [ ]:
samples1 = FLMC.FLA(potential1, alpha=1.7, N =10000, a_eta=1e-7, b_eta=0.6, step_size=0.03, dim=2)

indices = ~np.logical_or(np.isnan(samples1)[:, 0], np.isnan(samples1)[:, 1])
samples1 = samples1[indices]

plt.hist2d(samples1[:,0], samples1[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
# plt.xlim([-3.5, 3.5])
# plt.ylim([-3.5, 3.5])
plt.title('Empirical Density #1')
plt.show()

In [ ]:
def density2_np(z):
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return np.exp(-u)


def potential2(z):
    z = z.view(-1, 2).double()
    x, y = z[:, 0], z[:, 1]
    u = 0.8 * x ** 2 + (y - ((x**2)**(1/3)))**2
    u = u / 2**2
    return u

In [ ]:
r = np.linspace(-5, 5, 1000)
x, y = np.meshgrid(r, r)
z = np.vstack([x.flatten(), y.flatten()]).T

q0 = density2_np(z)
plt.pcolormesh(x, y, q0.reshape(x.shape),
                           cmap='viridis')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim([-3.5, 3.5])
plt.ylim([-3.5, 3.5])
plt.title('Density #2')
plt.show()

In [ ]:
samples2 = FLMC.FLA(potential2, alpha=1.7, N =10000, a_eta=1e-7, b_eta=0.6, step_size=0.03, dim=2)

indices = ~np.logical_or(np.isnan(samples2)[:, 0], np.isnan(samples2)[:, 1])
samples22 = samples2[indices]

plt.hist2d(samples2[:,0], samples2[:,1], cmap='viridis', rasterized=False, bins=200, density=True)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Empirical Density #2')
plt.show()